In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Change the plotting style
plt.style.use('seaborn-v0_8-whitegrid')

## =============================================
## 1. Core Gradient Descent Implementation
## =============================================

def gradient_descent(f, grad_f, x0, alpha_type='fixed', alpha=0.01, 
                    max_steps=1000, tolerance=0.0001, 
                    c1=1e-4, rho=0.5, max_line_search_iter=20):
    """
    Gradient descent optimization algorithm.
    
    Parameters:
    -----------
    f : function
        The objective function to minimize.
    grad_f : function
        The gradient of the objective function.
    x0 : numpy array
        The starting point.
    alpha_type : str
        Type of step size: 'fixed' or 'backtracking'.
    alpha : float
        Initial step size.
    max_steps : int
        Maximum number of iterations.
    tolerance : float
        Convergence tolerance.
    c1 : float
        Parameter for Armijo condition.
    rho : float
        Reduction factor for backtracking.
    max_line_search_iter : int
        Maximum backtracking iterations.
        
    Returns:
    --------
    X : numpy array
        Final solution point.
    f_values : list
        Function values at each iteration.
    path : list
        The optimization path.
    steps : int
        Number of steps taken.
    """
    X = np.array(x0, dtype='float64')
    path = [X.copy()]
    f_values = [f(X)]
    
    for step in range(max_steps):
        grad = grad_f(X)
        
        # Determine step size
        if alpha_type == 'backtracking':
            current_alpha = alpha
            for _ in range(max_line_search_iter):
                X_new = X - current_alpha * grad
                # Armijo condition
                if f(X_new) <= f(X) + c1 * current_alpha * np.dot(grad, -grad):
                    break
                current_alpha *= rho
            alpha_used = current_alpha
        else:
            alpha_used = alpha
        
        # Update position
        X_new = X - alpha_used * grad
        
        # Check for convergence
        if np.linalg.norm(X_new - X) < tolerance:
            break
            
        X = X_new
        path.append(X.copy())
        f_values.append(f(X))
    
    return X, f_values, path, step+1

## =============================================
## 2. Visualization Functions
## =============================================

def plot_function_3d(f, x_range=(-5, 5), y_range=(-5, 5), title=""):
    """Create a 3D surface plot of the function."""
    x = np.linspace(x_range[0], x_range[1], 100)
    y = np.linspace(y_range[0], y_range[1], 100)
    X, Y = np.meshgrid(x, y)
    Z = f([X, Y])
    
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, alpha=0.8)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('f(x,y)')
    ax.set_title(title)
    plt.show()

def plot_contour(f, x_range=(-5, 5), y_range=(-5, 5), title=""):
    """Create a contour plot of the function."""
    x = np.linspace(x_range[0], x_range[1], 100)
    y = np.linspace(y_range[0], y_range[1], 100)
    X, Y = np.meshgrid(x, y)
    Z = f([X, Y])
    
    plt.figure(figsize=(10, 6))
    plt.contour(X, Y, Z, levels=50)
    plt.colorbar()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title)
    plt.show()

def plot_convergence_path(f, path, title, x_range=(-5, 5), y_range=(-5, 5)):
    """Plot the optimization path on a contour plot."""
    x = np.linspace(x_range[0], x_range[1], 100)
    y = np.linspace(y_range[0], y_range[1], 100)
    X, Y = np.meshgrid(x, y)
    Z = f([X, Y])
    
    plt.figure(figsize=(10, 6))
    plt.contour(X, Y, Z, levels=50)
    plt.colorbar()
    
    # Plot path
    path_array = np.array(path)
    plt.plot(path_array[:, 0], path_array[:, 1], 'r.-', markersize=10)
    plt.scatter(path_array[0, 0], path_array[0, 1], c='green', s=100, label='Start')
    plt.scatter(path_array[-1, 0], path_array[-1, 1], c='blue', s=100, label='End')
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title)
    plt.legend()
    plt.show()

def plot_convergence_values(f_values, title):
    """Plot the function values during optimization."""
    plt.figure(figsize=(10, 6))
    plt.plot(f_values, 'b-', linewidth=2)
    plt.xlabel('Iteration')
    plt.ylabel('Function Value')
    plt.title(title)
    plt.grid(True)
    plt.show()

## =============================================
## 3. Test Functions and Their Gradients
## =============================================

# 1. Original function
def f1(X):
    x, y = X
    return (x-47)**2 + (y-0.1)**2 + 2

def grad_f1(X):
    x, y = X
    return np.array([2*(x-47), 2*(y-0.1)])

# 2. Rosenbrock function (n=5)
def rosenbrock(X):
    return sum(100*(X[i+1]-X[i]**2)**2 + (1-X[i])**2 for i in range(len(X)-1))

def grad_rosenbrock(X):
    n = len(X)
    grad = np.zeros(n)
    for i in range(n-1):
        grad[i] += -400*X[i]*(X[i+1]-X[i]**2) - 2*(1-X[i])
        grad[i+1] += 200*(X[i+1]-X[i]**2)
    return grad

# 3. Booth function
def booth(X):
    x, y = X
    return (x + 2*y - 7)**2 + (2*x + y - 5)**2

def grad_booth(X):
    x, y = X
    return np.array([
        2*(x + 2*y - 7) + 4*(2*x + y - 5),
        4*(x + 2*y - 7) + 2*(2*x + y - 5)
    ])

# 4. Himmelblau's function
def himmelblau(X):
    x, y = X
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

def grad_himmelblau(X):
    x, y = X
    return np.array([
        4*x*(x**2 + y - 11) + 2*(x + y**2 - 7),
        2*(x**2 + y - 11) + 4*y*(x + y**2 - 7)
    ])

# 5. Three-hump camel function
def camel(X):
    x, y = X
    return 2*x**2 - 1.05*x**4 + x**6/6 + x*y + y**2

def grad_camel(X):
    x, y = X
    return np.array([
        4*x - 4.2*x**3 + x**5 + y,
        x + 2*y
    ])

# 6. McCormick function
def mccormick(X):
    x, y = X
    return np.sin(x + y) + (x - y)**2 - 1.5*x + 2.5*y + 1

def grad_mccormick(X):
    x, y = X
    return np.array([
        np.cos(x + y) + 2*(x - y) - 1.5,
        np.cos(x + y) - 2*(x - y) + 2.5
    ])

## =============================================
## 4. Testing and Visualization
## =============================================

def test_function(f, grad_f, name, test_cases):
    """Test gradient descent on a function with various parameters."""
    print(f"\n===== Testing {name} Function =====")
    
    for i, case in enumerate(test_cases):
        print(f"\nTest Case {i+1}:")
        print(f"Initial point: {case['x0']}, Alpha: {case.get('alpha', 'backtracking')}")
        
        # Extraia apenas os parâmetros esperados pela gradient_descent
        valid_keys = ['x0', 'alpha_type', 'alpha', 'max_steps', 'tolerance', 'c1', 'rho', 'max_line_search_iter']
        gd_params = {k: v for k, v in case.items() if k in valid_keys}
        
        # Executa o gradiente descendente
        result = gradient_descent(f, grad_f, **gd_params)
        X_opt, f_values, path, steps = result
        
        print(f"Optimal point: {X_opt}")
        print(f"Steps: {steps}, Final value: {f_values[-1]}")
        
        # Se o ponto inicial for 2D, plota os resultados
        if len(case['x0']) == 2:
            plot_convergence_path(
                f, path, 
                f"{name} Function - Path (Case {i+1})",
                x_range=case.get('x_range', (-5,5)),
                y_range=case.get('y_range', (-5,5))
            )
            plot_convergence_values(
                f_values, 
                f"{name} Function - Convergence (Case {i+1})"
            )

# Define test cases for each function
test_cases_f1 = [
    {'x0': [80, 20], 'alpha_type': 'fixed', 'alpha': 0.5, 'x_range': (0,100), 'y_range': (-20,40)},
    {'x0': [80, 20], 'alpha_type': 'backtracking', 'alpha': 1.0, 'x_range': (0,100), 'y_range': (-20,40)}
]

test_cases_rosenbrock = [
    {'x0': np.zeros(5), 'alpha_type': 'fixed', 'alpha': 0.0001, 'max_steps': 10000},
    {'x0': np.ones(5), 'alpha_type': 'backtracking', 'alpha': 1.0, 'max_steps': 10000}
]

test_cases_booth = [
    {'x0': [0, 0], 'alpha_type': 'fixed', 'alpha': 0.01, 'x_range': (-10,10), 'y_range': (-10,10)},
    {'x0': [0, 0], 'alpha_type': 'fixed', 'alpha': 0.05, 'x_range': (-10,10), 'y_range': (-10,10)},
    {'x0': [0, 0], 'alpha_type': 'backtracking', 'alpha': 1.0, 'x_range': (-10,10), 'y_range': (-10,10)}
]

test_cases_himmelblau = [
    {'x0': [0, 0], 'alpha_type': 'fixed', 'alpha': 0.01, 'x_range': (-5,5), 'y_range': (-5,5)},
    {'x0': [3, 3], 'alpha_type': 'fixed', 'alpha': 0.01, 'x_range': (-5,5), 'y_range': (-5,5)},
    {'x0': [-3, -3], 'alpha_type': 'backtracking', 'alpha': 1.0, 'x_range': (-5,5), 'y_range': (-5,5)}
]

test_cases_camel = [
    {'x0': [1, 1], 'alpha_type': 'fixed', 'alpha': 0.01, 'x_range': (-2,2), 'y_range': (-2,2)},
    {'x0': [1, 1], 'alpha_type': 'backtracking', 'alpha': 1.0, 'x_range': (-2,2), 'y_range': (-2,2)}
]

test_cases_mccormick = [
    {'x0': [-1, 2], 'alpha_type': 'fixed', 'alpha': 0.01, 'x_range': (-1.5,4), 'y_range': (-3,4)},
    {'x0': [-1, 2], 'alpha_type': 'backtracking', 'alpha': 1.0, 'x_range': (-1.5,4), 'y_range': (-3,4)}
]

# Run tests
if __name__ == "__main__":
    test_function(f1, grad_f1, "Original", test_cases_f1)
    test_function(rosenbrock, grad_rosenbrock, "Rosenbrock", test_cases_rosenbrock)
    test_function(booth, grad_booth, "Booth", test_cases_booth)
    test_function(himmelblau, grad_himmelblau, "Himmelblau", test_cases_himmelblau)
    test_function(camel, grad_camel, "Camel", test_cases_camel)
    test_function(mccormick, grad_mccormick, "McCormick", test_cases_mccormick)
